In [0]:
%pip install rapidfuzz


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%restart_python

In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import mlflow
import mlflow.pyfunc


Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0xff6748587ba0>
Traceback (most recent call last):
  File "/databricks/python/lib/python3.12/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/databricks/python/lib/python3.12/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/python/lib/python3.12/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
                   ^^^^^^^^^^^^^^^^^^
  File "/databricks/python/lib/python3.12/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
             ^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'split'


In [0]:
df = spark.sql("SELECT * FROM realistic_inventory_dataset")
display(df)



PackedBy,BuiltBy,InstallBy,ItemType,OrderNo,QTY,Model,Brand,SERIALNo,Processor,RAM,HardDrive,OS,Cable,Comment,Courier,Dispatched,OrderID,PostCode,DispatchDate,Prebuilt_Or_Inventory
Chelsea,Kyle,Derek,Parts,ORD1001,5,16GB DDR4,Corsair,SN70601,null,null,null,null,No,Handle with care,DHL,No,OID1001,66517,2025-03-09,Prebuilt
Charles,Terri,Joseph,Desktop,ORD1002,4,Pavilion,HP,SN63350,Intel i7,32GB,1TB HDD,iOS,Yes,Handle with care,DHL,No,OID1002,762,2024-12-26,Inventory
Yolanda,Carolyn,Thomas,Desktop,ORD1003,1,ROG Strix,Asus,SN95724,Apple M1,4GB,2TB HDD,macOS,No,No comments,FedEx,Yes,OID1003,44884,2025-06-11,Inventory
Bradley,Jamie,Matthew,Parts,ORD1004,5,Ryzen 7 CPU,AMD,SN74574,null,null,null,null,Yes,Check hardware,DHL,Yes,OID1004,13548,2025-01-19,Prebuilt
Jessica,Nicholas,Benjamin,Parts,ORD1005,4,16GB DDR4,Corsair,SN35793,null,null,null,null,No,No comments,UPS,No,OID1005,88427,2025-01-25,Inventory
Julia,Mary,Caitlin,AIO,ORD1006,5,Vivo AIO,Asus,SN78799,AMD Ryzen 7,4GB,1TB HDD,Windows 10,Yes,Check hardware,FedEx,No,OID1006,11885,2025-06-28,Prebuilt
Laura,Angela,Benjamin,Laptop,ORD1007,2,Spectre x360,HP,SN49651,AMD Ryzen 7,32GB,1TB HDD,Ubuntu,No,No comments,DHL,Yes,OID1007,52119,2025-05-29,Inventory
Curtis,Kevin,Rhonda,AIO,ORD1008,4,Vivo AIO,Asus,SN57020,Intel i5,8GB,256GB SSD,Windows 10,Yes,Check hardware,DHL,No,OID1008,93067,2025-01-05,Inventory
Stephen,Scott,Tanner,AIO,ORD1009,3,iMac 24,Apple,SN28027,AMD Ryzen 7,32GB,512GB SSD,macOS,Yes,Handle with care,DHL,Yes,OID1009,61518,2025-02-11,Prebuilt
Jose,Mercedes,Jonathan,AIO,ORD1010,5,Vivo AIO,Asus,SN87510,AMD Ryzen 5,16GB,1TB HDD,Ubuntu,No,No comments,FedEx,No,OID1010,5604,2025-07-21,Inventory


In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from difflib import get_close_matches
import re

# -------------------------------
# 1️⃣ Convert PySpark DataFrame to Pandas
# -------------------------------
df_pandas = df.toPandas()

# -------------------------------
# 2️⃣ Handle categorical features
# -------------------------------
categorical_features = ['ItemType','Brand','Model','Processor','OS','Cable']
df_pandas[categorical_features] = df_pandas[categorical_features].fillna('Unknown')

# -------------------------------
# 3️⃣ Handle numerical features
# -------------------------------
numerical_features = ['RAM','HardDrive']

# RAM → numeric
df_pandas['RAM'] = df_pandas['RAM'].astype(str).str.replace('GB','', regex=False)
df_pandas['RAM'] = pd.to_numeric(df_pandas['RAM'], errors='coerce')

# HardDrive → numeric GB
def convert_hard_drive(value):
    if not value or pd.isna(value):
        return np.nan
    value = str(value).upper()
    number_match = re.search(r'\d+(\.\d+)?', value)
    if not number_match:
        return np.nan
    number = float(number_match.group())
    if 'TB' in value:
        return number * 1024
    return number  # assume GB if no TB

df_pandas['HardDrive'] = df_pandas['HardDrive'].apply(convert_hard_drive)

# Drop rows with missing numerical values
df_pandas = df_pandas.dropna(subset=numerical_features)

# Ensure integers safely
df_pandas['RAM'] = df_pandas['RAM'].fillna(0).astype(float).round().astype(int)
df_pandas['HardDrive'] = df_pandas['HardDrive'].fillna(0).astype(float).round().astype(int)

# -------------------------------
# 4️⃣ Encode & scale features
# -------------------------------
# One-hot encode categorical features
encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
cat_encoded = encoder.fit_transform(df_pandas[categorical_features])

# Standardize numerical features
scaler = StandardScaler()
num_scaled = scaler.fit_transform(df_pandas[numerical_features])

# Combine features
feature_vectors = np.hstack([cat_encoded, num_scaled])

# -------------------------------
# 5️⃣ Prepare product list
# -------------------------------
df_pandas['ProductName'] = (
    df_pandas['Brand'].astype(str) + " " +
    df_pandas['Model'].astype(str) + " " +
    df_pandas['ItemType'].astype(str) + " " +
    df_pandas['Processor'].astype(str) + " " +
    df_pandas['RAM'].astype(str) + "GB " +
    df_pandas['HardDrive'].astype(str) + "GB " +
    df_pandas['OS'].astype(str)
).str.lower()

product_list = df_pandas['ProductName'].tolist()

# -------------------------------
# 6️⃣ Compute cosine similarity
# -------------------------------
similarity_matrix = cosine_similarity(feature_vectors)

# -------------------------------
# 7️⃣ Top-N recommendation function (with explainability)
# -------------------------------
def recommend_products(product_name, top_n=5):
    # lowercase query for consistent matching
    product_name = product_name.lower()
    
    # fuzzy match
    matches = get_close_matches(product_name, product_list, n=1, cutoff=0.6)
    if not matches:
        return {"error": f"'{product_name}' not found in catalog"}
    
    matched_name = matches[0]
    idx = product_list.index(matched_name)
    
    # similarity scores
    sim_scores = list(enumerate(similarity_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # top recommendations with details
    recommendations = []
    for i, score in sim_scores[1:top_n+1]:  # skip itself
        rec = {
            "product": product_list[i],
            "similarity": round(score, 3),
            "RAM": df_pandas.iloc[i]["RAM"],
            "HardDrive": df_pandas.iloc[i]["HardDrive"],
            "Processor": df_pandas.iloc[i]["Processor"],
            "OS": df_pandas.iloc[i]["OS"]
        }
        recommendations.append(rec)
    
    return {"matched_name": matched_name, "recommendations": recommendations}

# -------------------------------
# 8️⃣ Example usage
# -------------------------------
example_product = "HP Pavilion Desktop"
print(f"Top recommendations for {example_product}:")
print(recommend_products(example_product, top_n=5))


Top recommendations for HP Pavilion Desktop:
{'matched_name': 'hp pavilion desktop apple m2 4gb 256gb ios', 'recommendations': [{'product': 'hp pavilion desktop apple m2 4gb 512gb windows 11', 'similarity': 0.861, 'RAM': 4, 'HardDrive': 512, 'Processor': 'Apple M2', 'OS': 'Windows 11'}, {'product': 'hp pavilion desktop apple m1 16gb 256gb ios', 'similarity': 0.849, 'RAM': 16, 'HardDrive': 256, 'Processor': 'Apple M1', 'OS': 'iOS'}, {'product': 'hp pavilion desktop amd ryzen 5 4gb 256gb ios', 'similarity': 0.749, 'RAM': 4, 'HardDrive': 256, 'Processor': 'AMD Ryzen 5', 'OS': 'iOS'}, {'product': 'hp pavilion desktop intel i7 8gb 256gb windows 11', 'similarity': 0.741, 'RAM': 8, 'HardDrive': 256, 'Processor': 'Intel i7', 'OS': 'Windows 11'}, {'product': 'apple imac 24 desktop apple m2 8gb 512gb ios', 'similarity': 0.723, 'RAM': 8, 'HardDrive': 512, 'Processor': 'Apple M2', 'OS': 'iOS'}]}


In [0]:
# -------------------------------
# 6️⃣ Compute cosine similarity
# -------------------------------
similarity_matrix = cosine_similarity(feature_vectors)
print("Cosine similarity matrix:")
print(similarity_matrix)


Cosine similarity matrix:
[[ 1.          0.11596493  0.10821144 ...  0.16251915  0.04048072
   0.20719896]
 [ 0.11596493  1.          0.26284491 ... -0.04127756  0.58146417
   0.17605423]
 [ 0.10821144  0.26284491  1.         ...  0.24022517  0.11382943
  -0.06076628]
 ...
 [ 0.16251915 -0.04127756  0.24022517 ...  1.          0.27677775
   0.17679253]
 [ 0.04048072  0.58146417  0.11382943 ...  0.27677775  1.
   0.41224382]
 [ 0.20719896  0.17605423 -0.06076628 ...  0.17679253  0.41224382
   1.        ]]


In [0]:
import joblib
import pickle
import os
import numpy as np
import pandas as pd  # ✅ make sure pandas is imported

# --- 9️⃣ Save Model Components ---

model_dir = "recommendation_model_artifacts"
os.makedirs(model_dir, exist_ok=True)

# Save encoder and scaler
joblib.dump(encoder, os.path.join(model_dir, "encoder.joblib"))
joblib.dump(scaler, os.path.join(model_dir, "scaler.joblib"))

# Save frozen similarity matrix (optional if you want static recommendations)
np.save(os.path.join(model_dir, "similarity_matrix.npy"), similarity_matrix)

# Save product list
with open(os.path.join(model_dir, "product_list.pkl"), "wb") as f:
    pickle.dump(product_list, f)

# ✅ Save the product features DataFrame for dynamic recommendations
columns_needed = [
    "ProductName", "ItemType", "Brand", "Model",
    "Processor", "OS", "Cable", "RAM", "HardDrive"
]

# Make sure we're working with pure Pandas DataFrame
df_features = pd.DataFrame(df_pandas[columns_needed].copy())

# Ensure correct dtypes
df_features["RAM"] = df_features["RAM"].astype(float)
df_features["HardDrive"] = df_features["HardDrive"].astype(float)

for col in ["ItemType", "Brand", "Model", "Processor", "OS", "Cable", "ProductName"]:
    df_features[col] = df_features[col].astype(str)

# Save as pickle (pure Pandas, no PySpark dependency)
df_features.to_pickle(os.path.join(model_dir, "product_features.pkl"))

print(f"✅ Recommendation model artifacts saved to '{model_dir}' directory.")


✅ Recommendation model artifacts saved to 'recommendation_model_artifacts' directory.
